# <center> ML-7. Прогнозирование биологического ответа (HW-3)
#### <center> Необходимо предсказать биологический ответ молекул (столбец *'Activity'*) по их химическому составу (столбцы *D1-D1776*).
---

[**&#8595; Скачать данные**](https://lms-cdn.skillfactory.ru/assets/courseware/v1/9f2add5bca59f8c4df927432d605fff3/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/_train_sem09__1_.zip "_train_sem09__1_.zip")

Данные представлены в формате CSV.  Каждая строка представляет молекулу. 

* Первый столбец *Activity* содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
* Остальные столбцы *D1-D1776* представляют собой молекулярные **дескрипторы** — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.


Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать **F1-score**.

Необходимо обучить две модели: **логистическую регрессию** и **случайный лес**. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать **все четыре метода** (*GridSeachCV*, *RandomizedSearchCV*, *Hyperopt*, *Optuna*) хотя бы по разу, максимальное количество итераций не должно превышать 50.

---

### Baseline

##### Предобработка

In [1]:
import numpy as np
import pandas as pd
import hyperopt
import optuna

from sklearn import model_selection
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics

In [2]:
data = pd.read_csv('data/_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data['Activity'].value_counts(True)

Activity
1    0.542255
0    0.457745
Name: proportion, dtype: float64

Дисбаланс классов небольшой. Но стратификацию всё равно лучше оставить.

In [4]:
X = data.drop('Activity', axis=1)
y = data['Activity']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.2)

##### Обучение с параметрами по умолчанию

Сначала обучим модели с параметрами по умолчанию кроме random_state и max_iter.

LogisticRegression

In [5]:
log_reg = linear_model.LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train, y_train)
y_test_pred_lr = log_reg.predict(X_test)
print('LogisticRegression test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr)))

LogisticRegression test f1-score: 0.78


RandomForestClassifier

In [6]:
rf_clf = ensemble.RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)
y_test_pred_rf = rf_clf.predict(X_test)
print('RandomForestClassifier test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf)))

RandomForestClassifier test f1-score: 0.80


Видно что обе модели даже с параметрами по умолчанию дают хорошие значения метрики. Случайный лес чуть лучше.

---
### Подбор гиперпараметров

##### GridSearchCV

LogisticRegression

In [7]:
# Задаём сетку параметров
# Т.к. разные алгоритмы поддерживают разные типы регуляризации, зададим два словаря
param_grid_lr = [
              {'penalty': ['l2', 'none'],
                'C': list(np.linspace(0.01, 1, 10, dtype=float)),
               'solver': ['lbfgs', 'saga']},
              
              {'penalty': ['l1', 'l2'],
                'C': list(np.linspace(0.01, 1, 10, dtype=float)),
               'solver': ['liblinear']}
             ]

In [8]:
grid_search_lr = model_selection.GridSearchCV(
    # Модель
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    # Сетка параметров
    param_grid=param_grid_lr, 
    # Количество фолдов для кросс-валидации
    cv=5,
    # Все доступные ядра
    n_jobs = -1
)

In [9]:
grid_search_lr.fit(X_train, y_train)
# Выводим лучшие значения гиперпараметров
print("Best parameters: {}".format(grid_search_lr.best_params_))
# Считаем f1-меру
y_test_pred_lr_gs = grid_search_lr.predict(X_test)
print('LogisticRegression GridSearchCV test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr_gs)))

Best parameters: {'C': 0.45, 'penalty': 'l1', 'solver': 'liblinear'}
LogisticRegression GridSearchCV test f1-score: 0.78


RandomForestClassifier

In [10]:
param_grid_rf = [
              {'n_estimators': list(np.linspace(100, 500, 5, dtype=int)),
                'criterion': ['gini', 'entropy'],
               'max_depth': list(np.linspace(5, 30, 6, dtype=int)),
               'min_samples_leaf': list(np.linspace(1, 30, 5, dtype=int))}
             ]

In [11]:
grid_search_rf = model_selection.GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid_rf, 
    cv=5,
    n_jobs = -1
)

In [12]:
grid_search_rf.fit(X_train, y_train)
print("Best parameters: {}".format(grid_search_rf.best_params_))

y_test_pred_rf_gs = grid_search_rf.predict(X_test)
print('RandomForestClassifier GridSearchCV test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf_gs)))

Best parameters: {'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 1, 'n_estimators': 200}
RandomForestClassifier GridSearchCV test f1-score: 0.81


Как видно, метрика на логистической регресси не изменилась, а на случайном лесе подросла незначительно. При этом время работы существенно вазросло.

---

##### RandomizedSearchCV

Будем использовать те же сетки параметров, что и в GridSearchCV.

LogisticRegression

In [13]:
param_distributions_lr = param_grid_lr

random_search_lr = model_selection.RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    param_distributions=param_distributions_lr, 
    cv=5,
    # В GridSearchCV было 60 комбинаций, попробуем оставить половину
    n_iter = 30, 
    n_jobs = -1
)
# Выводим значения оптимальных параметров и метрику
random_search_lr.fit(X_train, y_train)
print("Best parameters: {}".format(random_search_lr.best_params_))

y_test_pred_lr_rs = random_search_lr.predict(X_test)
print('LogisticRegression RandomizedSearchCV test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr_rs)))

Best parameters: {'solver': 'liblinear', 'penalty': 'l1', 'C': 0.34}
LogisticRegression RandomizedSearchCV test f1-score: 0.78


Метрика и набор параметров такие же, как в GridSearchCV, зато время существенно сократилось.

RandomForestClassifier

In [14]:
param_distributions_rf = param_grid_rf

random_search_rf = model_selection.RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions_rf, 
    cv=5,
    # Количество итераций поставим максимальное из возможных по условию
    n_iter = 50, 
    n_jobs = -1
)
# Выводим значения оптимальных параметров и метрику
random_search_rf.fit(X_train, y_train)
print("Best parameters: {}".format(random_search_rf.best_params_))

y_test_pred_rf_rs = random_search_rf.predict(X_test)
print('RandomForestClassifier RandomizedSearchCV test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf_rs)))

Best parameters: {'n_estimators': 200, 'min_samples_leaf': 1, 'max_depth': 20, 'criterion': 'entropy'}
RandomForestClassifier RandomizedSearchCV test f1-score: 0.80


Метрика немного просела по сравнению с GridSearchCV, видимо случайным образом не была выбрана оптимальная комбинация параметров, но время сократилось в разы.

---

##### Hyperopt

LogisticRegression

In [15]:
# Создадим списки категориальных параметров, чтобы можно было обратиться по индексам
penalty = ['l2', None]
solver = ['lbfgs', 'newton-cg', 'sag']
# Задаём пространство параметров
space_lr = {'penalty': hyperopt.hp.choice('penalty', penalty),
         'C': hyperopt.hp.quniform('C', 0.01, 1, 0.01),
         'solver': hyperopt.hp.choice('solver', solver)}
# фиксируем random_state
random_state=42

In [16]:
# Задаём целевую функцию
def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    
    # Создаём и обучаем модель
    model = linear_model.LogisticRegression(**params, random_state=random_state, max_iter=1000)
    model.fit(X, y)
    # Используем кросс-валидацию
    score = model_selection.cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score

In [17]:
# Логируем результаты
trials_lr = hyperopt.Trials() 

best_params_lr = hyperopt.fmin(hyperopt_lr,
          space=space_lr,
          algo=hyperopt.tpe.suggest,
          max_evals=20,
          trials=trials_lr, 
          rstate=np.random.default_rng(random_state)
         )

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(

c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 20%|██        | 4/20 [01:03<03:24, 12.75s/trial, best loss: -0.7874539264350459]

c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(

c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 25%|██▌       | 5/20 [01:17<03:17, 13.18s/trial, best loss: -0.7874539264350459]

...

 75%|███████▌  | 15/20 [08:00<02:19, 27.90s/trial, best loss: -0.7902171664645037]

c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(

c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 80%|████████  | 16/20 [08:12<01:33, 23.39s/trial, best loss: -0.7902171664645037]

c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(

c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



100%|██████████| 20/20 [10:29<00:00, 31.48s/trial, best loss: -0.7902171664645037]


Даёт много предупреждений, но если сильно увеличить max_iter, то время слишком сильно возрастёт.

In [18]:
# Создаём словарь с оптимальными параметрами
best_lr = {'C': best_params_lr['C'], 'penalty': penalty[best_params_lr['penalty']], 'solver': solver[best_params_lr['solver']]}
print("Best parameters: {}".format(best_lr))
# Обучаем модель с оптимальными параметрами
lr_ho = linear_model.LogisticRegression(
    **best_lr,
    random_state=random_state,
    max_iter=1000
)

lr_ho.fit(X_train, y_train)

y_test_pred_lr_ho = lr_ho.predict(X_test)

print('LogisticRegression Hyperopt test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr_ho)))

Best parameters: {'C': 0.05, 'penalty': 'l2', 'solver': 'lbfgs'}
LogisticRegression Hyperopt test f1-score: 0.79


RandomForestClassifier

In [19]:
criterion = ['gini', 'entropy']

space_rf = {'n_estimators': hyperopt.hp.quniform('n_estimators', 100, 500, 10),
            'criterion': hyperopt.hp.choice('criterion', criterion),
            'max_depth': hyperopt.hp.quniform('max_depth', 5, 30, 1),
            'min_samples_leaf': hyperopt.hp.quniform('min_samples_leaf', 2, 30, 1)}

In [20]:
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])}
    
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    model.fit(X, y)

    score = model_selection.cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score

In [21]:
trials_rf = hyperopt.Trials() 

best_params_rf = hyperopt.fmin(hyperopt_rf,
          space=space_rf,
          algo=hyperopt.tpe.suggest,
          max_evals=20,
          trials=trials_rf, 
          rstate=np.random.default_rng(random_state)
         )

100%|██████████| 20/20 [02:44<00:00,  8.23s/trial, best loss: -0.811118376235045]


In [22]:
best_rf = {'n_estimators': int(best_params_rf['n_estimators']), 
           'criterion': criterion[best_params_rf['criterion']], 
           'max_depth': int(best_params_rf['max_depth']), 
           'min_samples_leaf': int(best_params_rf['min_samples_leaf'])}
print("Best parameters: {}".format(best_rf))

rf_ho = ensemble.RandomForestClassifier(
    **best_rf,
    random_state=random_state
)

rf_ho.fit(X_train, y_train)

y_test_pred_rf_ho = rf_ho.predict(X_test)

print('LogisticRegression Hyperopt test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf_ho)))

Best parameters: {'n_estimators': 160, 'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 4}
LogisticRegression Hyperopt test f1-score: 0.80


У логистической регрессии метрика подросла, но при этом время выполнения для неё достаточно высокое. Метрика на случайном лесе осталась такой же, зато время выполнения существенно сократилась.

---

##### Optuna

LogisticRegression

In [23]:
def optuna_lr(trial):
    # Пространство гиперпараметров
    penalty = trial.suggest_categorical('penalty', ['l2', None])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'newton-cg', 'sag'])
    C = trial.suggest_float('C', 0.01, 1)
    # Модель
    model = linear_model.LogisticRegression(penalty=penalty,
                                            solver=solver,
                                            C=C,
                                            random_state=random_state,
                                            max_iter=1000)
    # Обучаем модель
    model.fit(X_train, y_train)
    # Считаем и возвращаем метрику
    score = model_selection.cross_val_score(model, X_train, y_train, cv=5, scoring="f1", n_jobs=-1).mean()
    return score

In [24]:
study_lr = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# Ищем комбинацию гиперпараметров на 20 итерациях
study_lr.optimize(optuna_lr, n_trials=20)

[I 2024-06-11 22:05:41,679] A new study created in memory with name: LogisticRegression
c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
[I 2024-06-11 22:06:26,067] Trial 0 finished with value: 0.7127612076011095 and parameters: {'penalty': None, 'solver': 'newton-cg', 'C': 0.6203703931216903}. Best is trial 0 with value: 0.7127612076011095.
c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-package

В очередной раз проделаем то же самое: выведем оптимальные значения гиперпараметров, обучим модель с их исплользованием и посчитаем метрику на тестовой выборке.

In [25]:
print("Best parameters: {}".format(study_lr.best_params))

lr_op = linear_model.LogisticRegression(
    **study_lr.best_params,
    random_state=random_state,
    max_iter=1000
)

lr_op.fit(X_train, y_train)

y_test_pred_lr_op = lr_op.predict(X_test)

print('LogisticRegression Optuna test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr_op)))

Best parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.019403441910238253}
LogisticRegression Optuna test f1-score: 0.78


RandomForestClassifier

In [26]:
def optuna_rf(trial):

    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 30)
    
    model = ensemble.RandomForestClassifier(criterion=criterion,
                                            n_estimators=n_estimators,
                                            max_depth=max_depth,
                                            min_samples_leaf=min_samples_leaf,
                                            random_state=random_state)
    model.fit(X_train, y_train)

    score = model_selection.cross_val_score(model, X_train, y_train, cv=5, scoring="f1", n_jobs=-1).mean()
    return score

In [27]:
study_rf = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
study_rf.optimize(optuna_rf, n_trials=20)

[I 2024-06-11 22:13:40,919] A new study created in memory with name: RandomForestClassifier
[I 2024-06-11 22:13:48,924] Trial 0 finished with value: 0.7874080893415613 and parameters: {'criterion': 'entropy', 'n_estimators': 284, 'max_depth': 27, 'min_samples_leaf': 21}. Best is trial 0 with value: 0.7874080893415613.
[I 2024-06-11 22:13:58,290] Trial 1 finished with value: 0.7948048311237698 and parameters: {'criterion': 'gini', 'n_estimators': 321, 'max_depth': 15, 'min_samples_leaf': 17}. Best is trial 1 with value: 0.7948048311237698.
[I 2024-06-11 22:14:05,266] Trial 2 finished with value: 0.773998669833714 and parameters: {'criterion': 'gini', 'n_estimators': 299, 'max_depth': 16, 'min_samples_leaf': 30}. Best is trial 1 with value: 0.7948048311237698.
[I 2024-06-11 22:14:14,923] Trial 3 finished with value: 0.7953717513744054 and parameters: {'criterion': 'gini', 'n_estimators': 325, 'max_depth': 12, 'min_samples_leaf': 15}. Best is trial 3 with value: 0.7953717513744054.
[I 202

In [28]:
print("Best parameters: {}".format(study_rf.best_params))

rf_op = ensemble.RandomForestClassifier(
    **study_rf.best_params,
    random_state=random_state)

rf_op.fit(X_train, y_train)

y_test_pred_rf_op = rf_op.predict(X_test)

print('RandomForestClassifier Optuna test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf_op)))

Best parameters: {'criterion': 'entropy', 'n_estimators': 384, 'max_depth': 21, 'min_samples_leaf': 2}
RandomForestClassifier Optuna test f1-score: 0.80


Метрика на логистической регрессии слегка снизилась по сравнению с Hyperopt, но время немного уменьшилось. а для случайного леса наоборот чуть-чуть возросло.

---

В целом Optuna представляется наболее удобной, т.к. и задание параметров сделано логичнее и вывод представлен не индексами, а значениями, в отличие от Hyperopt. Но, судя по всему, достаточно большое влияние на итоговый результат оказывает задание изначального пространства параметров.